# Preços de imóveis em João Pessoa

## Introdução

Esta análise se baseia na raspagem de dados na OLX para a pesquisa <a href = "https://www.olx.com.br/brasil?q=im%C3%B3veis%20em%20jo%C3%A3o%20pessoa&o=1">imóveis em joão pessoa</a>, seguida de análise exploratória desses dados.

São empregadas para tanto as seguintes bibliotecas:

- Requests para a realização de requisições HTTP;

- time para intervalos entre requisições;

- random para randomizar intervalos(otimização);

- BeautifulSoup para a extração de dados do html das páginas;

- re para a filtragem mais fina desses dados;

- Pandas para manipulação de dados em dataframes.

## 1. Importar bibliotecas

In [1]:
import requests
import time
from random import uniform
from bs4 import BeautifulSoup
import re
import pandas as pd

## 2. Web scraping

A primeira parte de nossa análise será a extração de dados por scraping do site da OLX. Para tanto procederemos assim:

1. Inicialmente varremos as páginas de pesquisa à procura dos links de cada um dos produtos;

2. Depois, para cada produto, faremos a raspagem de suas informações em sua própria página.

In [3]:
base_url = 'https://www.olx.com.br/brasil?q=im%C3%B3veis%20em%20jo%C3%A3o%20pessoa'

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

products_links = []
for page in range(1, 2):
    
    url = f"{base_url}&o={page}"

    try:
        search_response = requests.get(url, headers = headers)

        if search_response.status_code == 200:
            print(f"Processando página {page}...")
            search_soup = BeautifulSoup(search_response.text, 'html.parser')

            all_links = search_soup.find_all('a', class_ = 'olx-adcard__link')
            if all_links:
                for link in all_links:
                    href = link.get('href')
                    if href and href.startswith('http'):
                        products_links.append(href)

        else:
            print(f"Erro ao tentar acessar página {page}: {search_response.status.code}")

    except Exception as e:
        print(f"Erro: {e}")

    print(f"Produtos processados! Total de produtos nesta página: {len(all_links)}")

    time.sleep(uniform(1,3))

Processando página 1...
Produtos processados! Total de produtos nesta página: 50


Vejamos os dez primeiros links que coletamos:

In [11]:
products_links[0:10]

['https://pb.olx.com.br/paraiba/imoveis/imoveis-na-planta-em-joao-pessoa-1427390473',
 'https://pb.olx.com.br/paraiba/imoveis/vendo-imoveis-de-alto-padrao-a-beira-mar-de-intermares-1399963055',
 'https://pb.olx.com.br/paraiba/imoveis/imoveis-na-planta-1424880641',
 'https://pb.olx.com.br/paraiba/imoveis/imoveis-exclusivos-1422666330',
 'https://pb.olx.com.br/paraiba/servicos/imoveis-vende-ou-aluga-casa-apartamento-terreno-1348330489',
 'https://pb.olx.com.br/paraiba/imoveis/alugo-imoveis-1418639984',
 'https://pb.olx.com.br/paraiba/imoveis/apartamentos-para-venda-varios-imoveis-1425816782',
 'https://pb.olx.com.br/paraiba/servicos/reforma-de-imoveis-em-joao-pessoa-pisos-pintura-e-outros-acabamentos-1424786399',
 'https://pb.olx.com.br/paraiba/servicos/corretor-de-imoveis-em-joao-pessoa-1357257740',
 'https://pb.olx.com.br/paraiba/servicos/seu-corretor-de-imoveis-perito-avaliador-em-joao-pessoa-1385905983']

São exatamente os dez links que aparecem na primeira página de busca! Vamos agora investigá-los individualmente:

In [7]:
all_products = []

for i, link in enumerate(products_links, 1):
    try:
        print(f"Processando produto {i}/{len(products_links)}...")
        
        product_response = requests.get(link, headers=headers, timeout=10)

        if product_response.status_code == 200:
            product_soup = BeautifulSoup(product_response.text, 'html.parser')

            title_elem = product_soup.find('span', class_='olx-text olx-text--title-medium olx-text--block ad__sc-1l883pa-2 bdcWAn')
            title = title_elem.text.strip() if title_elem else 'Não especificado'

            location_div = product_soup.find('div', id='location')
            if location_div:
                location_elem = location_div.find('span', class_='olx-text olx-text--body-small olx-text--block olx-text--semibold olx-color-neutral-110')
                location = location_elem.text.strip() if location_elem else 'Não especificado'
            else:
                location = 'Não especificado'
            
            price_div = product_soup.find('div', id='price-box-container')
            if price_div:
                price_elem = price_div.find('span', class_='olx-text olx-text--title-medium olx-text--block')
                price = price_elem.text.strip() if price_elem else 'Não especificado'
                
                product_type_elem = price_div.find('span', class_='olx-badge olx-badge--neutral olx-badge--small olx-badge--rectangle')
                product_type = product_type_elem.text.strip() if product_type_elem else "Não especificado"
            else:
                price = 'Não especificado'
                product_type = "Não especificado"

            product_data = {
                'title': title,
                'location': location,
                'price': price,
                'product_type': product_type,
                'url': link  
            }

            all_products.append(product_data)
            print(f"Produto {i} extraído: {title}")

        else:
            print(f"Erro ao acessar produto {i}: Status {product_response.status_code}")

    except Exception as e:
        print(f"Erro no produto {i}: {e}")
    
    time.sleep(uniform(2, 4))

print(f"Extração concluída! Total de produtos: {len(all_products)}")

Processando produto 1/50...
Produto 1 extraído: Imóveis na planta em João Pessoa
Processando produto 2/50...
Produto 2 extraído: Vendo Imóveis de Alto padrão a Beira Mar de Intermares
Processando produto 3/50...
Produto 3 extraído: Imóveis na Planta
Processando produto 4/50...
Produto 4 extraído: Imoveis exclusivos
Processando produto 5/50...
Produto 5 extraído: IMOVEIS VENDE OU ALUGA CASA APARTAMENTO TERRENO
Processando produto 6/50...
Produto 6 extraído: ALUGO IMÓVEIS
Processando produto 7/50...
Produto 7 extraído: APARTAMENTOS PARA VENDA - VÁRIOS IMÓVEIS
Processando produto 8/50...
Produto 8 extraído: Reforma de Imóveis em João Pessoa: pisos, pintura e outros acabamentos
Processando produto 9/50...
Produto 9 extraído: Corretor de Imóveis em João Pessoa
Processando produto 10/50...
Produto 10 extraído: Seu Corretor de Imóveis / Perito Avaliador em João Pessoa
Processando produto 11/50...
Produto 11 extraído: Vistoria de imóveis
Processando produto 12/50...
Produto 12 extraído: Corret

In [8]:
all_products

[{'title': 'Imóveis na planta em João Pessoa',
  'location': 'Cabo Branco, João Pessoa, PB, 58045010',
  'price': 'Não especificado',
  'product_type': 'Venda',
  'url': 'https://pb.olx.com.br/paraiba/imoveis/imoveis-na-planta-em-joao-pessoa-1427390473'},
 {'title': 'Vendo Imóveis de Alto padrão a Beira Mar de Intermares',
  'location': 'Ipês, João Pessoa, PB, 58028530',
  'price': 'Não especificado',
  'product_type': 'Venda',
  'url': 'https://pb.olx.com.br/paraiba/imoveis/vendo-imoveis-de-alto-padrao-a-beira-mar-de-intermares-1399963055'},
 {'title': 'Imóveis na Planta',
  'location': 'Jardim Oceania, João Pessoa, PB, 58037030',
  'price': 'Não especificado',
  'product_type': 'Venda',
  'url': 'https://pb.olx.com.br/paraiba/imoveis/imoveis-na-planta-1424880641'},
 {'title': 'Imoveis exclusivos',
  'location': 'Manaíra, João Pessoa, PB, 58038142',
  'price': 'Não especificado',
  'product_type': 'Venda',
  'url': 'https://pb.olx.com.br/paraiba/imoveis/imoveis-exclusivos-1422666330'}